2.a) The plot of process model and the data on the same graph.

# The process model 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import norm

In [ ]:
def process(q,alpha):
    x_0 = np.random.normal(0,1)
    X = np.zeros(30)
    X[0] = x_0
    for i in range (1,30):
        delta = np.random.normal(0,q)
        X[i] = alpha * X[i-1] + delta
    return X

# The data model

In [ ]:
def data(X,r,T,T1,T2):
    n = len(X)
    Y = np.zeros(n)
    for i in range(0,n):
        epsilon = np.random.normal(0,r)
        if i in range(0,T1+1):
            Y[i]=X[i]+epsilon
        if i in range(T1,T2+1):
            Y[i]=0.1*X[i]+epsilon
        if i in range(T2,T+1):
            Y[i]=X[i]+epsilon
    return Y

alpha=0.8
q=0.4
r=0.1
T=30
T1=10
T2=20
X=process(alpha,q)
Y=data(X,r,T,T1,T2)

In [ ]:
plt.figure(figsize=(10,8), dpi = 65)
plt.plot(Y, 'o',color='red',label='Data')
plt.plot(X, color ='blue',label='Process')
plt.xlabel('time')
plt.legend()
plt.show()

The choise of h_i is affecting an increase of standard deviation of data model fron process model i.e the data model is not fitting the process model.

# b)Kalman filter to update the states x_i

In [ ]:
def Kalm(M,Q,R,H,y,Sgm,Mu):
    N = len(y)
    mu_old = np.zeros(N)
    sigma_old = np.zeros(N)
    mu_new = np.zeros(N+1)
    mu_new[0] = Mu
    sigma_new = np.zeros(N+1)
    sigma_new[0] = Sgm
    for t in range(1,N+1):
        mu_old[t-1] = M*mu_new[t-1]
        sigma_old[t-1] = Q + M*sigma_new[t-1]*M
        k = sigma_old[t-1]*H*(H*sigma_old[t-1]*H + R)**-1
        mu_new[t] = mu_old[t-1] + k*(y[t-1] - H*mu_old[t-1])
        sigma_new[t] = (1 - k*H)*sigma_old[t-1] 
    return mu_old, mu_new[1:], sigma_old, sigma_new[1:],k

q=0.4
r=0.1
alp=0.8
Mu =0
Sgm=1
Q = q**2
R = r**2
H = 1
mu_old,mu_new,sigma_old,sigma_new,k = Kalm(alp,Q,R,H,Y,Sgm,Mu)

In [ ]:
def fun(mu_old,mu_new,sigma_old,sigma_new,y):
    N = len(y)
    Xforecast = np.zeros(N)
    Xfilter = np.zeros(N)
    for t in range(0,N):
        Xforecast[t] = np.random.normal(mu_old[t],np.sqrt(sigma_old[t]))
        Xfilter[t] = np.random.normal(mu_new[t],np.sqrt(sigma_new[t]))
    return Xforecast , Xfilter

In [ ]:
Xforecast,Xfilter = fun(mu_old,mu_new,sigma_old,sigma_new,Y)
plt.figure(figsize=(8,6), dpi = 80)
plt.plot(Y,'o', label = 'Data', color='black')
plt.plot(X,label = 'Process', color ='blue')
plt.plot(Xforecast, label = 'Forecasting', color ='red')
plt.plot(Xfilter, label = 'Filtering', color='green')
plt.legend()
plt.show()

For figure above we can see that there is no  much noise between data model and process model and we can see also that there is no big biases between filtering and process model while there is big bias between process model and forecasting

# c)Standard deviation of the filtered and forecasted estimates together with their errors

In [ ]:
plt.figure(figsize=(8,6), dpi = 85)
plt.plot(np.sqrt(sigma_old)+mu_old,label='Forecast_Upper',color='red') 
plt.plot(mu_old,label='Forecast_Mean',color='m')
plt.plot(mu_old-np.sqrt(sigma_old),label='Forecast_Lower',color='g')
plt.legend()
plt.show()

The figure above shows that the biases between forecast lower and mean is not much large except between 10 and 20 x-axis
and also for forecast upper the biases between it and mean is seems to be like the ones on forecast lower.

In [ ]:
plt.figure(figsize=(12,8), dpi = 85)
plt.plot(np.sqrt(sigma_new[1:])+mu_new[1:],label='Filter_Upper',color='red') 
plt.plot(mu_new[1:],label='Filter_Mean',color='m')
plt.plot(mu_new[1:]-np.sqrt(sigma_new[1:]),label='Filter_Lower',color='g')
plt.legend()
plt.show()

From the figure above we can see that the biases between filter upper and filter lower is not large except between 10 and 20.

# 3.a) The one dimensional nonlinear process model

__Process model__

In [ ]:
def PROCESS(alp,q):
    x_0 = np.random.normal(0.1,0.01)
    X = np.zeros(60)
    X[0] = x_0
    for i in range (1,30):
        delta = np.random.normal(0,q)
        X[i] = alp * X[i-1]*(1-X[i-1]) + delta
    return X

__Data model__

In [ ]:
def DATA(X,r):
    n = len(X)
    Y = np.zeros(n)
    for i in range(0,n):
        ep = np.random.normal(0,r)
        Y[i]=X[i]+ep
    return Y

__Parameters__

In [ ]:
alp=0.9;q=0.001;r=0.005

__Plot__

In [ ]:
N_X=PROCESS(alp,q)
N_Y=DATA(N_X,r)
plt.figure(figsize=(8,8), dpi = 85)
plt.plot(N_X,color ='black',label='Process')
plt.plot(N_Y,'o',color='blue',label='Data')
plt.legend()
plt.show()

__b. Kalman filter__

Approximation function

In [ ]:
def APPROXIMATION(a_0,PROCESS):
    N=len(PROCESS)
    M_1=np.zeros(N)
    for i in range(N):
        M_1[i]=a_0*(1-2*PROCESS[i])
    return M_1

In [ ]:
def KALMAN_EXT(mu_0,sig_0,M,M_1,Q,R,H,y):
    N = len(y)
    mu = np.zeros(N)
    sig = np.zeros(N)
    mu_new = np.zeros(N+1)
    mu_new[0] = mu_0
    sig_new = np.zeros(N+1)
    sig_new[0] = sig_0
    for t in range(1,N+1):
        mu[t-1] = M*mu_new[t-1]
        sig[t-1] = Q + M_1[t-1]*sig_new[t-1]*M_1[t-1]
        k = sig[t-1]*H*(H*sig[t-1]*H + R)**-1
        mu_new[t] = mu[t-1] + k*(y[t-1] - H*mu[t-1])
        sig_new[t] = (1 - k*H)*sig[t-1] 
    return mu, mu_new[1:], sig, sig_new [1:]

In [ ]:
def ESTIMATION(mu,mu_new,sig,sig_new,y):
    N = len(y)
    X_FORE = np.zeros(N)
    X_FILT = np.zeros(N)
    for t in range(0,N):
        X_FORE[t] = np.random.normal(mu[t],np.sqrt(sig[t]))
        X_FILT[t] = np.random.normal(mu_new[t],np.sqrt(sig_new[t]))
    return X_FORE , X_FILT

q=0.001;r=0.005;mu_0 =0.1;sig_0=0.01;Q = q**2;R = r**2;H = 1;a=0.9

N_X1=PROCESS(a,q)

M_1=APPROXIMATION(a,N_X1)
mu,mu_new,sig,sig_new= KALMAN_EXT(mu_0,sig_0,a,M_1,Q,R,H,N_Y)


In [ ]:
N_XFORE,N_XFILT = ESTIMATION(mu,mu_new,sig,sig_new,N_Y)

In [ ]:
plt.figure(figsize=(8,8), dpi = 85)
plt.plot(N_Y,'o', label = 'Data', color='blue')
plt.plot(N_X,label = 'Process', color ='black')
plt.plot(N_XFILT, label = 'Filtering', color='red')
plt.plot(N_XFORE, label = 'Forecasting', color ='cyan')
plt.xlabel('Time')
plt.legend()
plt.show()

From figure above we can see that standard deviation between process model and data model is not large and we can see that the standard devition between filtering and process model, data modelis small and even for forecast has small deviation from model except at 3 there is an outlier.

# c) Plot of standard deviation of both filtered and forecasted estimates

In [ ]:
plt.figure(figsize=(12,8),dpi=85)
plt.plot(np.sqrt(sig)+mu,label='Forecast_Upper',color ='black')
plt.plot(mu,label='Mean',color ='blue')
plt.plot(mu-np.sqrt(sig),label='Forecast_Lower',color ='green')
plt.legend()
plt.show()

The figure above shows that the biases between forecast lower and mean is not much large except between 0 on  x-axis and also for forecast upper the biases between it and mean is seems to be like the ones on forecast lower.

In [ ]:
plt.figure(figsize=(12,8),dpi=85)
plt.plot(np.sqrt(sig_new[1:])+mu_new[1:],label='Filter_Upper',color ='black')
plt.plot(mu_new[1:],label='Mean',color ='blue')
plt.plot(mu_new[1:]-np.sqrt(sig_new[1:]),label='Filter_Lower',color ='red')
plt.legend()
plt.show()

The figure above shows that the biases between forecast lower and mean is not much large except from 14 x-axis and also for forecast upper the biases between it and mean is seems to be like the ones on forecast lower.